In [2]:
import pandas as pd
from sentence_transformers import (
    SentenceTransformer,
    SentencesDataset,
    losses,
    CrossEncoder,
)
from torch.utils.data import DataLoader
from sentence_transformers.readers import InputExample

# sbert.org

import utils


In [3]:
dataset_path = utils.get_global_vars()["dataset_path"]
train_df = pd.read_csv(f"{dataset_path}/balanced_unhashed_train_df.csv")
test_df = pd.read_csv(f"{dataset_path}/balanced_unhashed_test_df.csv")


In [4]:
train_df


,comment_text,comment_author,comment_score,answer1_text,answer1_author,answer1_score,answer2_text,answer2_author,answer2_score,subreddit,answer1_label,answer2_label
0,I've been searching for the answer for this fo...,hans1193,194,Religion must have the answer,Reedzit,3,It's obviously tracks from a giant water tract...,adaminc,1,science,1,0
1,"Michael Phelps Apologizes For ""Regrettable"" Be...",meg1,1,Wow...he smoked pot...oh lord hes such a horri...,rodneykm,1,"Wow, his girlfriend is uhm... Ah fuck it, he's...",dash101,2,news,1,0
2,Utah wants to create a database to track the i...,Clintondiditfirst,21,I think the government should track every morm...,monkeyman114,1,Another idea from the party that wants to get ...,mellowmonk,0,reddit.com,0,1
3,The Six Million Dead Jews of World War ONE!,[deleted],0,"Oh right, *both* wars were just jewish conspir...",Erobern,6,"i know this seems strange but, what if he was ...",five5,1,politics,1,0
4,WSJ begins the Jeb Bush campaign for 2016,[deleted],14,Good luck with that.,Mastrmind,2,time to get that shack in montana.,[deleted],1,politics,1,0
...,...,...,...,...,...,...,...,...,...,...,...,...
9995,Horn of Africa drought: 'A vision of hell',florence25,118,God works in mysterious ways.,Hyperian,-1,"""We were supposed to have an early warning sys...",bopollo,0,worldnews,1,0
9996,Look what I found at a local flea market,tswaves,232,"Wait, did you just get a DS for 20 bucks?",Ruskin,6,I'll buy it for $19.50,Darkmatter010,11,gaming,0,1
9997,"Will Kirk Cameron shut the fuck up, now?",VladMcRad,102,that's just proof of jesus's awesome creativit...,White_kimbo,2,Now will Kirk Cameron explain why why there ar...,danxmason,2,atheism,1,0
9998,Thank goodness for sunglasses,juuwaaaan,91,Sounds like you should spend more time saving ...,nullx,1,"When making funny pie charts, always make sure...",KnifeFed,2,funny,1,0


In [15]:
from tqdm.notebook import tqdm

model = CrossEncoder("microsoft/deberta-base", num_labels=2)

train_examples = []
# i = 0
for index, row in tqdm(list(train_df.iterrows())):
    # if i > 3:
    #     break
    # i += 1
    # print(f"row['comment_text'] = '{row['comment_text']}'")
    # print(f"row['answer1_text'] = '{row['answer1_text']}'")
    # print(f"row['answer2_text'] = '{row['answer2_text']}'")
    # print(f"row['answer1_label'] = '{row['answer1_label']}'")
    # print(f"row['answer2_label'] = '{row['answer2_label']}'")
    # print("-" * 40)

    x = InputExample(
        texts=[row["comment_text"], row["answer1_text"]],
        label=str(row["answer1_label"])
    )
    train_examples.append(x)
    x = InputExample(
        texts=[row["comment_text"], row["answer2_text"]],
        label=str(row["answer2_label"])
    )
    train_examples.append(x)

train_dataset = SentencesDataset(train_examples, model)
train_dataloader = DataLoader(train_dataset, shuffle=True, batch_size=16)


Some weights of the model checkpoint at microsoft/deberta-base were not used when initializing DebertaForSequenceClassification: ['lm_predictions.lm_head.LayerNorm.bias', 'lm_predictions.lm_head.dense.weight', 'lm_predictions.lm_head.LayerNorm.weight', 'lm_predictions.lm_head.dense.bias', 'lm_predictions.lm_head.bias']
- This IS expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model trained on another task or with another architecture (e.g. initializing a BertForSequenceClassification model from a BertForPreTraining model).
- This IS NOT expected if you are initializing DebertaForSequenceClassification from the checkpoint of a model that you expect to be exactly identical (initializing a BertForSequenceClassification model from a BertForSequenceClassification model).
Some weights of DebertaForSequenceClassification were not initialized from the model checkpoint at microsoft/deberta-base and are newly initialized: ['pooler.dense.weight', 'pooler

  0%|          | 0/10000 [00:00<?, ?it/s]

In [7]:
train_examples

In [16]:
model.fit(
    train_dataloader=train_dataloader,
    # evaluator=evaluator,
    epochs=5,
    warmup_steps=100,
)

/mnt/e/Lucas/_Estudos IME-USP/7_Semestre/Trabalho_de_Conclusão_de_Curso_MAC0499/Projeto/venv/lib/python3.10/site-packages/transformers/optimization.py:306: FutureWarning: This implementation of AdamW is deprecated and will be removed in a future version. Use the PyTorch implementation torch.optim.AdamW instead, or set `no_deprecation_warning=True` to disable this warning
  warnings.warn(


Epoch:   0%|          | 0/5 [00:00<?, ?it/s]

Iteration:   0%|          | 0/1250 [00:00<?, ?it/s]

ValueError: too many dimensions 'str'